In [ ]:
# Code to visualise the Fourier Decomposition of a plucked string
# Don't run all the cells simulatenously, run them one after another

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 200

from matplotlib.animation import FuncAnimation

In [ ]:
# A set of functions which, given certain parameters (location of plucking (a), height (h), a string-position vector (x)) computes different coefficients
def plucked(a, h, x):                                       # Function produces the initial condition on the string, a linear ramp, first up, then down
    f0 = np.zeros(len(x))
    L = max(x)
    
    f0[x<=a] = h/a * x[x<=a]
    f0[x>=a]  = h/2 - h/(L-a) * ( x[x>=a] - (L+a)/2)
    
    return f0

def plucked_coefficients(n,a,h,x):                          # Function that returns the coefficient of the plucked string's Fourier series
    L = max(x)
    return (2/L)*h*(-L *np.sin(a*n*np.pi) + a * np.sin(L*n*np.pi))/(a*(a-L)*n**2 *np.pi**2)


def mode_with_time(t,n,a,h,x, with_actual_amplitude=True): # A single normal mode with time dependence. Can either return the mode with a constant amplitude of 0.4
    L = max(x)                                              # or the actual amplitude (in terms of its contribution to the Fourier series)
    if(with_actual_amplitude):
        return (2/L)*h*(-L *np.sin(a*n*np.pi) + a * np.sin(L*n*np.pi))/(a*(a-L)*n**2 *np.pi**2)*np.sin(n*np.pi*x/L)*np.cos(n*np.pi*t/L)
    else:
        return 0.4*np.sin(n*np.pi*x/L)*np.cos(n*np.pi*t/L)

def summed_coefficients(N,a,h,x):                           # Recursive function that returns the summed coefficients up to some N
    L = max(x)
    if(N==0):
        return 0
    
    return plucked_coefficients(N,a,h,x)*np.sin(N*np.pi*x/L) + summed_coefficients(N-1,a,h,x)

def summed_coefficients_with_time(t,N,a,h,x):               # Recursive function that returns the summed coefficients with time dependence up to some N. 
    L = max(x)                                              # This is just an approximation of the full time-dependent solution
    if(N==0):
        return 0
    
    return plucked_coefficients(N,a,h,x)*np.sin(N*np.pi*x/L)*np.cos(N*np.pi*t/L) + summed_coefficients_with_time(t,N-1,a,h,x)

In [ ]:
####### Plot the initial condition for different numbers of terms in the Fourier Series, just to see if it's working
x = np.linspace(0,1,1000)
alpha=0.9
plt.plot(x, plucked(0.2,0.5,x), color='black',  label=r"f(x)", ls='dashdot',lw=2, alpha=1)
plt.plot(x, summed_coefficients(1, 0.2,0.5,x),  label=r"$N=1$ term", ls='solid',color='tab:blue', alpha=alpha)
plt.plot(x, summed_coefficients(3, 0.2,0.5,x),  label=r"$N=3$ terms", ls='solid',color='firebrick', alpha=alpha)
plt.plot(x, summed_coefficients(10, 0.2,0.5,x), label=r"$N=10$ terms", ls='solid',color='darkgoldenrod', alpha=alpha)
plt.plot(x, summed_coefficients(30, 0.2,0.5,x), label=r"$N=30$ terms", ls='solid',color='black', alpha=alpha)
ylim = plt.ylim()
plt.vlines(0.2,ylim[0],ylim[1], color='purple', ls='--')
plt.gca().set_aspect(1)
plt.legend()

In [ ]:
%%time
#### Solve the problem for 6 different different Ns (this takes a little time to run, around 30 seconds)

T = 10                                                       # Total time
n_steps = 5000                                               # Number of time-steps

soln = np.zeros((6,n_steps, len(x)))                         # Array to store the full solution for each N, for each time-step, for each point
modes= np.zeros((6,n_steps, len(x)))                         # Array to store each mode N with actual amplitude, for each time-step, for each point
modes_with_constant_amplitude= np.zeros((6,n_steps, len(x))) # Array to store each mode N with a `normalised' amplitude, for each time-step for each point
times = np.linspace(0,T,n_steps)                             # Array of time-steps

Ns = [1,2,3,4,5,6]                                           # Different numbers of modes

for j in range(len(Ns)):                                     # For each Ns and each instant of time, get the configuration of the string and save it in an array
    for i in range(len(times)):
        soln[j][i] = summed_coefficients_with_time(times[i], Ns[j],0.2,0.5,x) 
        modes[j][i] = mode_with_time(times[i], Ns[j],0.2,0.5,x) 
        modes_with_constant_amplitude[j][i] = mode_with_time(times[i], Ns[j],0.2,0.5,x, with_actual_amplitude=False) 

In [ ]:
# Defining a function to show animated plots, given an input array, and a couple of options. 
# The plot_titles option decides what the titles on each of the subplots will be. You can also
# choose to save the animation, providing the name, dpi, and fps. The filename can specify exact 
# output folder, as well as the filetype.

def show_animated_plots(array, plot_titles = "individual", save_animation = False, save_name="video.mp4", save_dpi = 300, save_fps=100):
    
    if(plot_titles != "individual" and plot_titles != "cumulative"):
        raise ValueError('The plot_titles variable can be either "cumulative" or "individual", no other options are currently supported')
    
    fig, axes = plt.subplots(nrows=2, ncols=3) # Create plot with 2 rows and 3 columns

    colors = [['tab:blue', 'firebrick', 'darkgoldenrod'], ['olive','navy', 'black']]  # Default colours for plots

    Nlabel = [[1,2,3],[4,5,6]]  # Labels for N per plot

    plots = []                  # Array to actually store the plot objects

    for row in range(len(axes)):                                          # Make the plots array a 1D array of len(Ns) items, 
        for col in range(len(axes[row])):                                 # each one being an individual plot object
            ln, = axes[row][col].plot([], [], color=colors[row][col])
            plots.append(ln)

    def init():                                                           # Initialise the whole plot, set x and y limits, titles, and axes 
        for row in range(len(axes)):
            for col in range(len(axes[row])):
                axes[row][col].set_xlim(0, 1)
                axes[row][col].set_ylim(-0.5, 0.5)
                if(plot_titles == "cumulative"):
                    axes[row][col].set_title(r"Sum of "+str(Nlabel[row][col])+ " terms", color=colors[row][col]) 
                elif(plot_titles == "individual"):
                    axes[row][col].set_title(r"Mode "+str(Nlabel[row][col]), color=colors[row][col]) 

                axes[row][col].tick_params(top=False, bottom=False, left=False, right=False,                   # Remove all axes (they're unnecessary here)
                    labelleft=False, labelbottom=False)
        return plots

    def animate(frame):                                  # For frame number `frame',
        xdata = x
        for i in range(len(Ns)):                         # for each N,
            ydata = array[i][frame]                      # for every time-step, get the string's position data, and
            plots[i].set_data(xdata, ydata)              # set each of the Ns plot to show the function at this step,
        return plots                                     # and return all the plots to the FuncAnimation function

    # Most important line, this is what actually handles the animation by calling the `animate' function with a frame number (integer)
    ani = FuncAnimation(fig, animate, init_func=init, blit=True, frames=1000, interval=1, repeat=True)   # Code to create animations
    
    if(save_animation):                                  # If you wish to save the animation, do so with the dpi and fps set here 
        ani.save(save_name, dpi=save_dpi, fps=save_fps)

    return ani                                           # When calling this FuncAnimation function from with a code, all the information that update the window 
                                                         # are attributes of the object ani. If you do not keep a reference to it around, then ani is garbage collected 
                                                         # all information about the graphs disappears when calling from within a function. You don't need this when you call 
                                                         # aren't calling the animation from within a function.

In [ ]:
plt.close()                                          # Close any previous plots
%matplotlib notebook

show_animated_plots(soln, plot_titles="cumulative")

In [ ]:
plt.close()                                          # Close any previous plots
%matplotlib notebook
show_animated_plots(modes, plot_titles="individual")

In [ ]:
plt.close()                                          # Close any previous plots
%matplotlib notebook
show_animated_plots(modes_with_constant_amplitude, plot_titles="individual")